# Post campaign
Пример анализа аудитории рекламной кампании.

## Описание задачи

Посчитаем аудиторию рекламных кампаний для бренда __Х__ за май-июнь 2020 года.

Общие параметры:
- Период: 1 мая - 30 июня 2020 года
- География: Россия 100 000+
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop, Web Mobile, App Online, App Offline).

Статистики:
- Unweighted Reach (UnwReach)
- Reach
- Reach% (ReachPer)
- Reach N Distribution 1-2 (ReachN)
- Reach N Distribution 3-4 (ReachN)
- Reach N Distribution 5+ (ReachN)


## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

### Общие параметры для задания

Для начала зададим общие параметры

In [ ]:
# задаем параметры
# выбираем тип установки mobile, т.к. в расчетах участвуют все типы устройств (Web Desktop, Web Mobile, App Online, App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-06-30'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Получим ID бренда
Для построения отчета нам необходимо получить идентификатор бренда, для этого воспользуемся ноутбуком, в котором приведены примеры работы с [каталогами](catalogs.ipynb).

## Задание
Перейдем к формированию задания.

### Задание №1. Аудитория рекламных кампаний


In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'х'

# в медиа фильтр нужно задать идентификатор вашего бренда
media_filter = "(brand = )"

# фильтр по демографии отсутствует
demo_filter = None

# задаем список статистик для расчета
statistics = ["UnwReach", "Reach", "ReachPer", "ReachN"]

# так как в отчете указана статистика ReachN, зададим для неё шкалу:
reach_n = [
         {"from": 1, "to": 2},
         {"from": 3, "to": 4},
         {"from": 5, "to": 255}
          ]

# указываем порядок группировки по брендам и позициям 
structure =  {
    "usetype": False,
    "media": ["brand", "position"]
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure, reach_n=reach_n)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_project = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_result = rtask.rename_reachn_columns(reach_n, df_project)
df_result

## Экспорт в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('post-campaign-Х'))
df_info = rtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()